In [ ]:
import pandas as pd
import numpy as np
import re
import json
from string import hexdigits
from pprint import pprint

In [ ]:
fname = '/home/welling/Downloads/Split upload uuid tracking.txt'

## some utilities

In [ ]:
def is_uuid(s):
    return(len(s) == 32 and all(c in hexdigits for c in s))
def is_hubmap_id(s):
    return(len(s) == 15 and s.startswith('HBM'))

# Parser for a .txt file downloaded from the "Split upload uuid tracking" doc

In [ ]:
def create_rec_generator(fname):
    with open(fname) as f:
        for line in f:
            yield line

class Tokenizer():
    def __init__(self, fname):
        self.tok_l = []
        self.rec_num = 0
        self.rec_gen = create_rec_generator(fname)
    def next_line(self):
        self.tok_l += reversed(self.rec_gen.__next__().split())
        self.rec_num += 1
    def get(self):
        try:
            while not self.tok_l:
                self.next_line()
            rslt = self.tok_l.pop()
            #print(f'get: {self.tok_l} <{rslt}>')
            return rslt
        except StopIteration:
            return None
    def pushback(self,tok):
        if tok != '':
            self.tok_l.append(tok)
            #print(f'pushback <{tok}> -> {self.tok_l}')

# tokenizer test
#tzr = Tokenizer(multi_file)
#while True:
#    tok = tzr.get()
#    if tok is None:
#        break

hits = []
state = 'empty'
currently_being_decomposed = None
children_of_this_decomposition = []
found_groups = []
tzr = Tokenizer(fname)
dbg = False
dbg_ct = 100
tok = None
while True:
    prev_tok = tok
    tok = tzr.get()
    #if tok and "bbdda69f7d867e20486f66ac2ebbd5b4" in tok:
    #   dbg = True
    if dbg:
        if dbg_ct > 0:
            print(f'top: state=<{state}> tok=<{tok}> currently_being_decomposed=<{currently_being_decomposed}>')
            dbg_ct -= 1
        else:
            dbg = False
    if tok is None:
        break
    elif tok == '':
        #print('skip space')
        pass
    elif state in ["empty"]:
        if tok.lower() in ["decomposition", "decomposing"]:
            if children_of_this_decomposition is not None:
                this_group = {
                    "parent": currently_being_decomposed,
                    "children": children_of_this_decomposition
                }
                children_of_this_decomposition = []
                pprint(this_group)
                found_groups.append(this_group)
            if tok.lower() in ["decomposition"]:
                state = "decomp"
            elif tok.lower() in ["decomposing"]:
                state = "of"
            else:
                print(f'exiting with state {state} tok {tok} prev_tok {prev_tok}')
                raise RuntimeError("Bad parse")
        else:
            pass
    elif state == "decomp":
        if tok.lower() in ["of"]:
            state = "of"
        else:
            pass
    elif state == "of":
        if is_uuid(tok):
            currently_being_decomposed = tok
            print(f'currently_being_decomposed <- {currently_being_decomposed}')
            state = "inner"
        else:
            pass
    elif state == "inner":
        if tok in ["->"]:
            state = "arrow"
        elif tok.lower() in ["decomposition", "decomposing"]:
            this_group = {
                "parent": currently_being_decomposed,
                "children": children_of_this_decomposition
            }
            children_of_this_decomposition = []
            pprint(this_group)
            found_groups.append(this_group)
            if tok.lower() in ["decomposition"]:
                state = "decomp"
            elif tok.lower() in ["decomposing"]:
                state = "of"
            else:
                print(f'exiting with state {state} tok {tok} prev_tok {prev_tok}')
                raise RuntimeError("Bad parse")
        else:
            pass
    elif state == "arrow":
        if is_uuid(tok):
            children_of_this_decomposition.append(tok)
        state = "inner"

if state == "inner" and currently_being_decomposed is not None:
    this_group = {
        "parent": currently_being_decomposed,
        "children": children_of_this_decomposition
    }
    pprint(this_group)
    found_groups.append(this_group)

        
#    if tzr.rec_num > 100:
#        break

#pprint(hits)

In [ ]:
recs = []
for group in found_groups:
    parent = group['parent']
    for elt in group['children']:
        recs.append({'uuid':elt, 'parent':parent})
df = pd.DataFrame(recs)
df.head()

In [ ]:
df.to_csv('/tmp/all_products_of_split_uploads.tsv', sep='\t')